In [58]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [59]:
import torch





In [60]:
pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
! pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
from hazm import *
from transformers import BertModel, BertConfig

In [61]:
normalizer = Normalizer()

In [62]:
text = "تیم استقلال  ثابت کرد از پرسپولیس کمتر نیست."
normalized_text = normalizer.normalize(text)

In [64]:
import torch
from transformers import BertModel, BertConfig, BertTokenizer

class SentimentModel(nn.Module):
    def __init__(self, config):
        super(SentimentModel, self).__init__()
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, input_ids, attention_mask, token_type_ids):
        _, pooled_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            return_dict=False
        )
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

# Step 1: Load the model architecture and tokenizer
labels = ['Beinolmelal', 'Eghtesadi', 'Ejtemaee', 'ElmiVaDaneshgahi', 'FarhangHonarVaResane', 'Siasi', 'Varzeshi']
label2id = {label: i for i, label in enumerate(labels)}
id2label = {v: k for k, v in label2id.items()}

config = BertConfig.from_pretrained(
    'HooshvareLab/bert-fa-base-uncased', **{
        'label2id': label2id,
        'id2label': id2label,
    })
tokenizer = BertTokenizer.from_pretrained('HooshvareLab/bert-fa-base-uncased')

# Step 2: Initialize the model
model = SentimentModel(config)

# Step 3: Load the model's state dictionary
model_state_dict = torch.load('/content/gdrive/MyDrive/pytorch_model.bin')
model.load_state_dict(model_state_dict)

# Step 4: Preprocess the input text
text = "می‌توان به دروغ گفت که وضعیت بورس، نفت و اقتصاد در این کشور رو به بهبود است."
normalized_text = normalizer.normalize(text)
inputs = tokenizer(normalized_text, padding=True, truncation=True, return_tensors="pt")

# Step 5: Make predictions
with torch.no_grad():
    model.eval()
    logits = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], token_type_ids=inputs['token_type_ids'])
    predicted_labels = torch.argmax(logits, dim=-1)
    predicted_labels = predicted_labels.item()  # Convert tensor to scalar value

    predicted_label_text = id2label[predicted_labels]
    print(predicted_label_text)




Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Siasi


In [35]:
"""class SentimentModel(nn.Module):

    def __init__(self, config):
        super(SentimentModel, self).__init__()

        self.bert = BertModel.from_pretrained('HooshvareLab/bert-fa-base-uncased')
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, input_ids, attention_mask, token_type_ids):
        _, pooled_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            return_dict=False)

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits"""

"class SentimentModel(nn.Module):\n\n    def __init__(self, config):\n        super(SentimentModel, self).__init__()\n\n        self.bert = BertModel.from_pretrained('HooshvareLab/bert-fa-base-uncased')\n        self.dropout = nn.Dropout(config.hidden_dropout_prob)\n        self.classifier = nn.Linear(config.hidden_size, config.num_labels)\n\n    def forward(self, input_ids, attention_mask, token_type_ids):\n        _, pooled_output = self.bert(\n            input_ids=input_ids,\n            attention_mask=attention_mask,\n            token_type_ids=token_type_ids,\n            return_dict=False)\n\n        pooled_output = self.dropout(pooled_output)\n        logits = self.classifier(pooled_output)\n        return logits"

In [36]:
"""config = BertConfig.from_pretrained('HooshvareLab/bert-fa-base-uncased')
model = SentimentModel(config)
model.load_state_dict(model_state_dict)"""

"config = BertConfig.from_pretrained('HooshvareLab/bert-fa-base-uncased')\nmodel = SentimentModel(config)\nmodel.load_state_dict(model_state_dict)"

In [37]:
"""import torch.nn as nn
if isinstance(model, nn.ModuleDict):
    model = model['model_key']
with torch.no_grad():
    model.eval()
    predicted_labels = model(text)"""

"import torch.nn as nn\nif isinstance(model, nn.ModuleDict):\n    model = model['model_key']\nwith torch.no_grad():\n    model.eval()\n    predicted_labels = model(text)"